# Data Visualisation of various metrics of Countries    


Imports


In [1]:
import pandas as pd
import numpy as np
import altair as alt
from altair import datum
import geopandas as gpd
from vega_datasets import data

Importing the csv as dataframe

In [2]:
# Load the data
df = pd.read_excel('world-data-2023_new.xlsx')

# print column names
print(df.columns)

Index(['Country', 'id', 'Abbreviation', 'Latitude', 'Longitude',
       'Density\n(P/Km2)', 'Agricultural Land( %)', 'Land Area(Km2)',
       'Armed Forces size', 'Birth Rate', 'Calling Code', 'Capital/Major City',
       'Co2-Emissions', 'CPI', 'CPI Change (%)', 'Currency-Code',
       'Fertility Rate', 'Forested Area (%)', 'Gasoline Price', 'GDP',
       'Gross primary education enrollment (%)',
       'Gross tertiary education enrollment (%)', 'Infant mortality',
       'Largest city', 'Life expectancy', 'Maternal mortality ratio',
       'Minimum wage', 'Official language', 'Out of pocket health expenditure',
       'Physicians per thousand', 'Population',
       'Population: Labor force participation (%)', 'Tax revenue (%)',
       'Total tax rate', 'Unemployment rate', 'Urban_population'],
      dtype='object')


In [3]:
#  print column names with corresponding null values
print(df.isnull().sum())

Country                                       0
id                                            0
Abbreviation                                  7
Latitude                                      1
Longitude                                     1
Density\n(P/Km2)                              0
Agricultural Land( %)                         7
Land Area(Km2)                                1
Armed Forces size                            24
Birth Rate                                    6
Calling Code                                  1
Capital/Major City                            3
Co2-Emissions                                 7
CPI                                          17
CPI Change (%)                               16
Currency-Code                                15
Fertility Rate                                7
Forested Area (%)                             7
Gasoline Price                               20
GDP                                           2
Gross primary education enrollment (%)  

Updating the dataset by adding continent and sub region columns, replacing missing values for the 2 letter codes according to ISO 3166 standard using country IDs

In [4]:
# loading the new dataset as a dataframe https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv
iso_data = pd.read_csv('ISO-3166.csv')

# selecting required columns
iso_data = iso_data[['country-code', 'region', 'sub-region', 'alpha-2', 'alpha-3']] 

# merging the two dataframes
df = pd.merge(df, iso_data, left_on='id', right_on='country-code', how='left')  

viewing the updated dataset

In [5]:
df.head()

,Country,id,Abbreviation,Latitude,Longitude,Density\n(P/Km2),Agricultural Land( %),Land Area(Km2),Armed Forces size,Birth Rate,...,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban_population,country-code,region,sub-region,alpha-2,alpha-3
0,Afghanistan,4,AF,33.939110,67.709953,60,0.581,652230.0,323000.0,32.49,...,0.489,0.093,0.714,0.1112,9797273.0,4,Asia,Southern Asia,AF,AFG
1,Albania,8,AL,41.153332,20.168331,105,0.431,28748.0,9000.0,11.78,...,0.557,0.186,0.366,0.1233,1747593.0,8,Europe,Southern Europe,AL,ALB
2,Algeria,12,DZ,28.033886,1.659626,18,0.174,2381741.0,317000.0,24.28,...,0.412,0.372,0.661,0.1170,31510100.0,12,Africa,Northern Africa,DZ,DZA
3,Andorra,20,AD,42.506285,1.521801,164,0.400,468.0,NaN,7.20,...,NaN,NaN,NaN,NaN,67873.0,20,Europe,Southern Europe,AD,AND
4,Angola,24,AO,-11.202692,17.873887,26,0.475,1246700.0,117000.0,40.73,...,0.775,0.092,0.491,0.0689,21061025.0,24,Africa,Sub-Saharan Africa,AO,AGO


dropping repeated columns

In [6]:
df.drop(columns=['id', 'Abbreviation'], inplace=True)
df.columns

Index(['Country', 'Latitude', 'Longitude', 'Density\n(P/Km2)',
       'Agricultural Land( %)', 'Land Area(Km2)', 'Armed Forces size',
       'Birth Rate', 'Calling Code', 'Capital/Major City', 'Co2-Emissions',
       'CPI', 'CPI Change (%)', 'Currency-Code', 'Fertility Rate',
       'Forested Area (%)', 'Gasoline Price', 'GDP',
       'Gross primary education enrollment (%)',
       'Gross tertiary education enrollment (%)', 'Infant mortality',
       'Largest city', 'Life expectancy', 'Maternal mortality ratio',
       'Minimum wage', 'Official language', 'Out of pocket health expenditure',
       'Physicians per thousand', 'Population',
       'Population: Labor force participation (%)', 'Tax revenue (%)',
       'Total tax rate', 'Unemployment rate', 'Urban_population',
       'country-code', 'region', 'sub-region', 'alpha-2', 'alpha-3'],
      dtype='object')

checking if any added columns have missing values

In [7]:
# checking if any added columns have missing values
df[['alpha-2', 'alpha-3', 'region', 'sub-region']].isnull().sum()

alpha-2       1
alpha-3       0
region        0
sub-region    0
dtype: int64

In [8]:
# printing out the country code and name for the country with missing alpha-2 code
print(df[df['alpha-2'].isnull()][['Country', 'country-code', 'alpha-3']])

     Country  country-code alpha-3
119  Namibia           516     NAM


In [9]:
#  adding the missing alpha-2 code from https://www.iban.com/country-codes
df.loc[df['country-code'] == 516, 'alpha-2'] = 'NA'

#  checking if the missing alpha-2 code has been added
print(df[df['country-code'] == 516][['Country', 'country-code', 'alpha-2']])

     Country  country-code alpha-2
119  Namibia           516      NA


checking data types of columns

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 39 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Country                                    195 non-null    object 
 1   Latitude                                   194 non-null    float64
 2   Longitude                                  194 non-null    float64
 3   Density
(P/Km2)                            195 non-null    int64  
 4   Agricultural Land( %)                      188 non-null    float64
 5   Land Area(Km2)                             194 non-null    float64
 6   Armed Forces size                          171 non-null    float64
 7   Birth Rate                                 189 non-null    float64
 8   Calling Code                               194 non-null    float64
 9   Capital/Major City                         192 non-null    object 
 10  Co2-Emissions             

transforming some percentage and currency datatypes to make them compatible with vega altair

In [11]:
# multiplying all the columns with '%' into 100 to get the actual percentage value and converting the columns to float
percentage_columns = [col for col in df.columns if '%' in col]
for col in percentage_columns:
    df[col] = df[col] * 100
    df[col] = df[col].astype(float)

In [12]:
df[percentage_columns].head()

,Agricultural Land( %),CPI Change (%),Forested Area (%),Gross primary education enrollment (%),Gross tertiary education enrollment (%),Population: Labor force participation (%),Tax revenue (%)
0,58.1,2.3,2.1,104.0,9.7,48.9,9.3
1,43.1,1.4,28.1,107.0,55.0,55.7,18.6
2,17.4,2.0,0.8,109.9,51.4,41.2,37.2
3,40.0,NaN,34.0,106.4,NaN,NaN,NaN
4,47.5,17.1,46.3,113.5,9.3,77.5,9.2


In [13]:
# changing the columns with currency signs to float
currency_columns = ['GDP', 'Minimum wage', 'Gasoline Price']

# removing the currency signs and saving it as a float value
for col in currency_columns:
    df[col] = df[col].str.replace('$', '').str.replace(',', '').astype(float)



In [14]:
df[currency_columns].head()

,GDP,Minimum wage,Gasoline Price
0,1.910135e+10,0.43,0.70
1,1.527808e+10,1.12,1.36
2,1.699882e+11,0.95,0.28
3,3.154058e+09,6.63,1.51
4,9.463542e+10,0.71,0.97


checking if data types have been updated

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 39 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Country                                    195 non-null    object 
 1   Latitude                                   194 non-null    float64
 2   Longitude                                  194 non-null    float64
 3   Density
(P/Km2)                            195 non-null    int64  
 4   Agricultural Land( %)                      188 non-null    float64
 5   Land Area(Km2)                             194 non-null    float64
 6   Armed Forces size                          171 non-null    float64
 7   Birth Rate                                 189 non-null    float64
 8   Calling Code                               194 non-null    float64
 9   Capital/Major City                         192 non-null    object 
 10  Co2-Emissions             

Adding new features to the dataset using the existing features

- GDP per capita
- CO2 emissions per capita

Calculating GDP per Capita:

In [16]:

# Calculate GDP per Capita
df['GDP per Capita'] = df['GDP'] / df['Population']


# Display the updated dataframe
df['GDP per Capita'].head()


0      502.115487
1     5352.857411
2     3948.343279
3    40886.391162
4     2973.591160
Name: GDP per Capita, dtype: float64

calculating CO2 emissions per capita

In [17]:
# Calculate CO2 emissions per capita
df['CO2 Emissions per Capita'] = (df['Co2-Emissions'] * 1000) / df['Population']

# Display the updated dataframe
df['CO2 Emissions per Capita'].head()

0    0.227960
1    1.589242
2    3.484213
3    6.079697
4    1.090108
Name: CO2 Emissions per Capita, dtype: float64

printing the countries with missing values for GDP per capita and CO2 emissions per capita

In [18]:
# Countries with missing GDP per Capita
missing_gdp_per_capita = df[df['GDP per Capita'].isnull()]['Country']
print("Countries with missing GDP per Capita:")
print(missing_gdp_per_capita)

# Countries with missing CO2 Emissions per Capita
missing_co2_per_capita = df[df['CO2 Emissions per Capita'].isnull()]['Country']
print("\nCountries with missing CO2 Emissions per Capita:")
print(missing_co2_per_capita)

Countries with missing GDP per Capita:
73                       Vatican City
133    Palestinian National Authority
Name: Country, dtype: object

Countries with missing CO2 Emissions per Capita:
56                           Eswatini
73                       Vatican City
113                            Monaco
120                             Nauru
128                   North Macedonia
133    Palestinian National Authority
149                        San Marino
Name: Country, dtype: object


We will try to impute missing values from reliable sources if data is available

In [19]:
# Adding the missing CO2 emissions data for countries
df.loc[df['Country'] == 'Eswatini', 'CO2 Emissions per Capita'] = 0.9
df.loc[df['Country'] == 'Nauru', 'CO2 Emissions per Capita'] = 4.8
df.loc[df['Country'] == 'North Macedonia', 'CO2 Emissions per Capita'] = 4.1
df.loc[df['Country'] == 'Palestinian National Authority', 'CO2 Emissions per Capita'] = 0.6

Processing features

In [20]:
df['Unemployment rate'] = df['Unemployment rate']*100

converting all float point values to 3 decimal points

In [21]:
df = df.round(3)

checking countries with null values for the selected features in visualisation.

Selected features: ['Country', 'Population', 'GDP per Capita', 'Gross primary education enrollment (%)', 'Gross tertiary education enrollment (%)', \
                'Unemployment rate', 'CO2 Emissions per Capita', 'Life expectancy', 'Physicians per thousand', 'Fertility Rate', 'Infant mortality', \
                    'Maternal mortality ratio', 'Out of pocket health expenditure']

In [22]:
#  print countries with null values for selected columns
columns=['Country', 'Population', 'GDP per Capita', 'Gross primary education enrollment (%)', 'Gross tertiary education enrollment (%)', \
                'Unemployment rate', 'CO2 Emissions per Capita', 'Life expectancy', 'Physicians per thousand', 'Fertility Rate', 'Infant mortality', \
                    'Maternal mortality ratio', 'Out of pocket health expenditure']

for col in columns:
    print(df[df[col].isnull()][['Country', col]])
    

Empty DataFrame
Columns: [Country, Country]
Index: []
                            Country  Population
133  Palestinian National Authority         NaN
                            Country  GDP per Capita
73                     Vatican City             NaN
133  Palestinian National Authority             NaN
                            Country  Gross primary education enrollment (%)
21           Bosnia and Herzegovina                                     NaN
56                         Eswatini                                     NaN
73                     Vatican City                                     NaN
113                          Monaco                                     NaN
120                           Nauru                                     NaN
128                 North Macedonia                                     NaN
133  Palestinian National Authority                                     NaN
                            Country  Gross tertiary education enrollment (%)
3        

In [23]:
# https://tradingeconomics.com/country-list/unemployment-rate?continent=world
unemployment_rates = {'Lichenstein':1.4, 'Palesian National Authority': 31.1, 'Seychelles':3.2} 

# https://databank.worldbank.org/source/world-development-indicators/Series/SE.TER.ENRR
tertiary_enrollment = {'Andorra': 64.3, 'Palau': 34.6, 'San Marino': 51.1, 'Marshall Islands': 53.3, 'North Macedonia': 53.2} 

# https://data.worldbank.org/indicator/SE.PRM.ENRR?locations=c
primary_enrollment= {'Bosnia and Herzegovina': 87, 'Eswatini': 114, 'Nauru':101, 'North Macedonia': 97}

# https://ourworldindata.org/life-expectancy
life_expectancy = {'Eswatini': 64.1, 'Nauru': 62.1, 'North Macedonia': 77.4, 'Palestinian National Authority': 65.2, 'Tuvalu': 67.1, 'Andorra': 84, 'Monaco': 86.4} 

# https://www.cia.gov/the-world-factbook/field/infant-mortality-rate/country-comparison/
infant_mortality = {'Eswatini': 36.7, 'Liechtenstein': 3.9, 'Nauru': 7.6, 'North Macedonia': 7}

# https://www.cia.gov/the-world-factbook/field/total-fertility-rate/country-comparison/
fertility_rate = {'Eswatini': 2.37, 'Nauru': 2.55, 'North Macedonia': 1.53, 'Monaco': 1.54, 'Tuvalu': 2.78}

maternal mortality, physicians per thousand, out of pocket health expenditure not available for all the countries that have missing data, data for coountries available for other features was sourced from reputable data sources. We will now add them to our dataset.

In [24]:
# Adding the missing data for unemployment rates
for country, rate in unemployment_rates.items():
    df.loc[df['Country'] == country, 'Unemployment rate'] = rate

# Adding the missing data for tertiary enrollment
for country, enrollment in tertiary_enrollment.items():
    df.loc[df['Country'] == country, 'Gross tertiary education enrollment (%)'] = enrollment

# Adding the missing data for primary enrollment
for country, enrollment in primary_enrollment.items():
    df.loc[df['Country'] == country, 'Gross primary education enrollment (%)'] = enrollment

# Adding the missing data for life expectancy
for country, expectancy in life_expectancy.items():
    df.loc[df['Country'] == country, 'Life expectancy'] = expectancy

# Adding the missing data for infant mortality
for country, mortality in infant_mortality.items():
    df.loc[df['Country'] == country, 'Infant mortality'] = mortality

# Adding the missing data for fertility rate
for country, rate in fertility_rate.items():
    df.loc[df['Country'] == country, 'Fertility Rate'] = rate

Importing geojson and preprocessing it

In [27]:
url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
country_shapes = gpd.read_file(url)
country_shapes = country_shapes[['NAME', 'ISO_A3', 'geometry']]
country_shapes['alpha-3'] = country_shapes['ISO_A3']

# printing countries where alpha-3 code doesnt match with the normal dataframe
unmatched_codes = country_shapes[~country_shapes['alpha-3'].isin(df['alpha-3'])]

print(unmatched_codes)

                       NAME ISO_A3  \
2                 W. Sahara    ESH   
20             Falkland Is.    FLK   
21                   Norway    -99   
22                Greenland    GRL   
23   Fr. S. Antarctic Lands    ATF   
43                   France    -99   
45              Puerto Rico    PRI   
134           New Caledonia    NCL   
140                  Taiwan    TWN   
159              Antarctica    ATA   
160               N. Cyprus    -99   
167              Somaliland    -99   
174                  Kosovo    -99   

                                              geometry alpha-3  
2    POLYGON ((-8.66559 27.65643, -8.66512 27.58948...     ESH  
20   POLYGON ((-61.2 -51.85, -60 -51.25, -59.15 -51...     FLK  
21   MULTIPOLYGON (((15.14282 79.67431, 15.52255 80...     -99  
22   POLYGON ((-46.76379 82.62796, -43.40644 83.225...     GRL  
23   POLYGON ((68.935 -48.625, 69.58 -48.94, 70.525...     ATF  
43   MULTIPOLYGON (((-51.6578 4.15623, -52.24934 3....     -99  
45   POLYGON

In [26]:
df.to_csv('world-data-2023_cleaned.csv', index=False)

Missing values for GDP per capita were not found for the countries and Missing data for CO2 emissions per capita was found for some of the countries.

We will now move onto plotting the data using Altair. 

We have selected the following metrics to plot:
- GDP per capita
- CO2 emissions per capita
- Population
- Life expectancy
- Continent
- Life Expectancy
- Infant Mortality
- Fertility Rate
- Physicians per Thousand
- Maternal Mortality Ratio


We will create the following graphs:

1. Choropleth World Map:
   - Attributes: GDP per Capita.
   - Description: A world map where countries are color-coded based on GDP per capita.
   - Interaction:
      - Hover Tooltips: Display country name, GDP per capita, life expectancy, and CO₂ emissions per capita.
      - Click to Select: Clicking on a country highlights it in all other views.
      - Zoom and Pan: Users can zoom into regions of interest.
      - Purpose: Provides a global overview of economic well-being, facilitating comparison between countries and regions.

2. Scatter Plot: GDP per Capita vs. Life Expectancy
   - Attributes:
      - X-axis: GDP per Capita.
      - Y-axis: Life Expectancy.
   - Color Encoding: Continent.
   - Size Encoding: Population.
   - Description: Each point represents a country, showing the relationship between economic status and health outcomes.
   - Interaction:
      - Brushing: Selecting points to highlight corresponding countries on the map and other plots.
      - Hover Tooltips: Display detailed information about the country.
      - Purpose: Allows users to explore correlations and patterns between wealth and health.

3. Interactive Box Plot: CO₂ Emissions per Capita
   - Attributes: CO₂ Emissions per Capita.
   - Description: Summarizes the distribution of CO₂ emissions per capita across countries.
   - Interaction:
      - Selection: Users can select quartiles or specific ranges to highlight countries in other views.
      - Hover Tooltips: Provide statistical summaries (median, quartiles, outliers).
      - Purpose: Highlights environmental impact without overcrowding the visualization, enabling users to identify countries with unusually high or low emissions.

4. Parallel Coordinates Plot: Health Indicators
   - Attributes:
      - Life Expectancy
      - Infant Mortality
      - Fertility Rate
      - Physicians per Thousand
      - Maternal Mortality Ratio
   - Description: Displays multiple health metrics for each country, with each line representing a country.
   - Interaction:
      - Brushing: Users can select ranges on any axis to filter and highlight countries.
      - Linking: Selected countries are highlighted in all other views.
      - Hover Tooltips: Provide detailed information on each country's health metrics.
      - Purpose: Enables multivariate analysis of health indicators, revealing patterns and outliers.